## This notebook produces data used in Supplementary Table 1

In [1]:
# SPDX-License-Identifier: GPL-3.0-or-later

In [2]:
%matplotlib widget

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.stats import linregress
import matplotlib
font = {'size'   : 12}

matplotlib.rc('font', **font)
import conntility


# Samples cells from exemplary regions and gets scaling factors

In [4]:
import bluepysnap as bp
s = bp.Simulation('/gpfs/bbp.cscs.ch/project/proj83/scratch/home/tharayil/BlueRecording/examples/whiskerFlick/data/simulation/simulation_config.json')
circuit = s.circuit

In [5]:
nodes = circuit.nodes

In [6]:
# Loads a generator that produces the cell types for all cells in the subvolume

nFl = nodes.get('hex_O1',properties=['mtype'])


In [7]:

Fl = next(nFl)[1]

In [8]:
# Loads node ids for Layer 5 TPC:A cells from each of the above regions

Fl = np.array(list(Fl[Fl['mtype'].isin(['L2_IPC','L2_TPC:A','L2_TPC:B','L3_TPC:A','L3_TPC:C'])].index))[:,1].astype(int)



In [9]:
len(Fl) # Number of L2/3 PCs, used in Supplementary table 1

53049

In [10]:
import h5py

In [11]:
# Loads the weights file

coeffsFile = "/gpfs/bbp.cscs.ch/project/proj83/scratch/home/tharayil/BlueRecording/examples/whiskerFlick/electrodeFile/coeffs_eeg.h5"

coeffs = h5py.File(coeffsFile)

In [12]:
# Node ids from weights file
allNodeIds = coeffs['S1nonbarrel_neurons']['node_ids']

In [13]:
# Finds indices in the weights file of nodes in the selected regions

flIndex = np.where(np.isin(allNodeIds[:],Fl))[0]



In [14]:
np.random.seed(1776)

# Samples nodes from the selected regions

flIndex = np.sort(np.random.choice(flIndex,1000,replace=False))


FLshort = allNodeIds[:][flIndex]



In [15]:
# Finds offsets for the compartment weights for the selected nodes

flOffsets = [coeffs['S1nonbarrel_neurons']['offsets'][:][flIndex],coeffs['S1nonbarrel_neurons']['offsets'][:][flIndex+1]]



In [19]:
# Gets compartment weights for the selected nodes (shifted such that the minimum weight for each neuron is 0)

scaleFacFl = []

ranges = []


for i in range(len(flOffsets[0])):
    
    sfl = coeffs['electrodes']['S1nonbarrel_neurons']['scaling_factors'][flOffsets[0][i]:flOffsets[1][i]][:,0]
    sfl -= np.min(sfl)

    scaleFacFl.append( sfl )

    ranges.append(np.percentile(sfl,90) - np.percentile(sfl,10))


In [20]:
r = np.mean(ranges) # Range of weights, used in supplementary table 1